<a href="https://colab.research.google.com/github/aderdouri/EiCNAM/blob/master/Tutorials/Notebooks/myCudaTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
\documentclass{article}
\usepackage{amsmath}
\usepackage{amssymb}
\usepackage{hyperref}
\usepackage{graphicx}
\usepackage{listings}
\usepackage{xcolor}

\lstset{
  basicstyle=\ttfamily\small,
  keywordstyle=\color{blue},
  commentstyle=\color{green!60!black},
  stringstyle=\color{red},
  showstringspaces=false,
  breaklines=true,
  frame=single,
  numbers=left,
  numberstyle=\tiny\color{gray},
  captionpos=b,
}

\begin{document}

\title{Extended Binomial Tree for Option Pricing}
\author{}
\date{}
\maketitle

\section*{Theory}
The price of an option satisfies the following partial differential equation (PDE):
\[
\frac{\partial V}{\partial t} + \frac{1}{2} \sigma^2(S, t) S^2 \frac{\partial^2 V}{\partial S^2} + r S \frac{\partial V}{\partial S} - r V = 0,
\]
where:
\begin{itemize}
    \item $V$: Option price
    \item $S$: Underlying asset price
    \item $\sigma(S, t)$: Volatility, which is state-dependent
    \item $r$: Risk-free rate
\end{itemize}

The extended binomial tree discretizes this PDE with state-dependent volatility to approximate the option value.

\section*{Algorithm}
\begin{enumerate}
    \item Discretize the time to maturity $T$ into $N$ steps, with each step $\Delta t = \frac{T}{N}$.
    \item At each time step, compute the up ($u$) and down ($d$) factors based on state-dependent volatility:
    \[
    u(S, t) = e^{\sigma(S, t) \sqrt{\Delta t}}, \quad d(S, t) = \frac{1}{u(S, t)}.
    \]
    \item Compute the transition probabilities ensuring risk neutrality:
    \[
    p = \frac{e^{r \Delta t} - d(S, t)}{u(S, t) - d(S, t)}.
    \]
    \item Apply backward induction:
    \[
    V(S, t) = \max\left(P(S), e^{-r \Delta t} [p V(Su, t+1) + (1-p) V(Sd, t+1)]\right).
    \]
\end{enumerate}

\section*{Python Implementation}
\lstset{language=Python}
\begin{lstlisting}
import numpy as np
import matplotlib.pyplot as plt

def extended_binomial_tree(S0, K, T, r, sigma_func, N, option_type="put"):
    """
    Extended binomial tree for option pricing with state-dependent volatility.

    Args:
        S0: Initial stock price.
        K: Strike price.
        T: Time to maturity.
        r: Risk-free rate.
        sigma_func: Function for state-dependent volatility, sigma(S, t).
        N: Number of steps in the tree.
        option_type: "call" or "put".

    Returns:
        Option price.
    """
    dt = T / N  # Time step
    u = lambda S, t: np.exp(sigma_func(S, t) * np.sqrt(dt))  # Up factor
    d = lambda S, t: 1 / u(S, t)  # Down factor

    # Initialize asset prices at maturity
    prices = np.zeros((N + 1, N + 1))
    prices[0, 0] = S0
    for i in range(1, N + 1):
        for j in range(i + 1):
            prices[j, i] = S0 * u(prices[j, i - 1], i * dt) if j == 0 else prices[j - 1, i - 1] * d(prices[j - 1, i - 1], i * dt)

    # Initialize option values at maturity
    option_values = np.maximum(0, K - prices[:, N]) if option_type == "put" else np.maximum(0, prices[:, N] - K)

    # Backward induction
    for i in reversed(range(N)):
        for j in range(i + 1):
            p = (np.exp(r * dt) - d(prices[j, i], i * dt)) / (u(prices[j, i], i * dt) - d(prices[j, i], i * dt))
            continuation_value = np.exp(-r * dt) * (p * option_values[j, i + 1] + (1 - p) * option_values[j + 1, i + 1])
            exercise_value = K - prices[j, i] if option_type == "put" else prices[j, i] - K
            option_values[j, i] = max(continuation_value, exercise_value)

    return option_values[0, 0]

# Example: American Put Option with State-Dependent Volatility
S0 = 100  # Initial stock price
K = 100   # Strike price
T = 1.0   # Time to maturity (1 year)
r = 0.05  # Risk-free rate
sigma_func = lambda S, t: 0.2 + 0.1 * np.sin(t)  # Example state-dependent volatility
N = 100   # Number of steps

price = extended_binomial_tree(S0, K, T, r, sigma_func, N, option_type="put")
print(f"American put option price: {price:.4f}")

# Visualization of Volatility
time_points = np.linspace(0, T, 100)
volatilities = [sigma_func(S0, t) for t in time_points]

plt.figure(figsize=(10, 6))
plt.plot(time_points, volatilities, label="Volatility: $\\sigma(S, t)$")
plt.xlabel("Time (t)")
plt.ylabel("Volatility ($\\sigma$)")
plt.title("State-Dependent Volatility")
plt.legend()
plt.grid()
plt.show()
\end{lstlisting}

\end{document}
